# BUSINESS PROBLEM

A gaming company wants to create level-based customer definitions (persona) using customers' features.

# ABOUT DATASET

The data set contains the prices of the products sold by an international gaming company and some demographic information of the users who purchased these products.
In this data set, a user with demographic characteristics may have made more than one purchase.


# COLUMNS

* PRICE:Customer's Spending Amount
* SOURCE:The Type of device the Customer Uses
* SEX:Customer's Gender
* COUNTRY – Customer's Country
* AGE – Customer's Age


# PREPARING DATA

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)


In [2]:
df=pd.read_csv("/kaggle/input/personacsv/persona.csv")

In [3]:
def check_df(dataframe,head=5,tail=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### Tail #####################")
    print(dataframe.tail(head))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

In [4]:
check_df(df)

##################### Shape #####################
(5000, 5)
##################### Types #####################
PRICE       int64
SOURCE     object
SEX        object
COUNTRY    object
AGE         int64
dtype: object
##################### Head #####################
   PRICE   SOURCE   SEX COUNTRY  AGE
0     39  android  male     bra   17
1     39  android  male     bra   17
2     49  android  male     bra   17
3     29  android  male     tur   17
4     49  android  male     tur   17
##################### Tail #####################
      PRICE   SOURCE     SEX COUNTRY  AGE
4995     29  android  female     bra   31
4996     29  android  female     bra   31
4997     29  android  female     bra   31
4998     39  android  female     bra   31
4999     29  android  female     bra   31
##################### NA #####################
PRICE      0
SOURCE     0
SEX        0
COUNTRY    0
AGE        0
dtype: int64
##################### Quantiles #####################
        count     mean        std  

In [5]:
df["SOURCE"].nunique()
df["SOURCE"].unique()
df["SOURCE"].value_counts()

SOURCE
android    2974
ios        2026
Name: count, dtype: int64

In [6]:
df["PRICE"].nunique()
df["PRICE"].value_counts()

PRICE
29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: count, dtype: int64

In [7]:
df.groupby("COUNTRY").agg({"PRICE":"count"})
df.groupby("COUNTRY").agg({"PRICE":"sum"})
df.groupby("COUNTRY").agg({"PRICE":"mean"})

,PRICE
COUNTRY,
bra,34.327540
can,33.608696
deu,34.032967
fra,33.587459
tur,34.787140
usa,34.007264


In [8]:
agg_df=df.groupby(["COUNTRY","SOURCE","SEX","AGE"]).agg({"PRICE":"mean"}).sort_values(by="PRICE",ascending=False)

In [9]:
agg_df.head()

,,,,PRICE
COUNTRY,SOURCE,SEX,AGE,
bra,android,male,46,59.0
usa,android,male,36,59.0
fra,android,female,24,59.0
usa,ios,male,32,54.0
deu,android,female,36,49.0


In [10]:
agg_df.reset_index(inplace=True)

In [11]:
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE
0,bra,android,male,46,59.0
1,usa,android,male,36,59.0
2,fra,android,female,24,59.0
3,usa,ios,male,32,54.0
4,deu,android,female,36,49.0


In [12]:
bins=[18,23,30,40,70,df["AGE"].max()]
labels=['0_18','19_23', '24_30', '31_40', '41_70','>70']

In [13]:
agg_df["AGE_CAT"]=pd.cut(agg_df["AGE"],6,bins,labels=labels)

In [14]:
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT
0,bra,android,male,46,59.0,31_40
1,usa,android,male,36,59.0,24_30
2,fra,android,female,24,59.0,19_23
3,usa,ios,male,32,54.0,19_23
4,deu,android,female,36,49.0,24_30


# Defining new level-based customers (personas)

In [15]:
agg_df["customers_level_based"]=agg_df["COUNTRY"] +"_"+ agg_df["SOURCE"]+"_" +agg_df["SEX"]+"_"+agg_df["AGE_CAT"].astype("O")
agg_df["customers_level_based"]=agg_df["customers_level_based"].str.upper()


In [16]:
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT,customers_level_based
0,bra,android,male,46,59.0,31_40,BRA_ANDROID_MALE_31_40
1,usa,android,male,36,59.0,24_30,USA_ANDROID_MALE_24_30
2,fra,android,female,24,59.0,19_23,FRA_ANDROID_FEMALE_19_23
3,usa,ios,male,32,54.0,19_23,USA_IOS_MALE_19_23
4,deu,android,female,36,49.0,24_30,DEU_ANDROID_FEMALE_24_30


In [17]:
agg_df=agg_df[["customers_level_based","PRICE"]]

In [18]:
agg_df["SEGMENT"]=pd.qcut(agg_df["PRICE"],q=4,labels=["D","C","B","A"])

In [19]:
agg_df.tail()

,customers_level_based,PRICE,SEGMENT
343,USA_IOS_FEMALE_24_30,19.0,D
344,USA_IOS_FEMALE_19_23,19.0,D
345,CAN_ANDROID_FEMALE_19_23,19.0,D
346,FRA_ANDROID_MALE_0_18,19.0,D
347,DEU_ANDROID_MALE_19_23,9.0,D


In [20]:
agg_df.head()

,customers_level_based,PRICE,SEGMENT
0,BRA_ANDROID_MALE_31_40,59.0,A
1,USA_ANDROID_MALE_24_30,59.0,A
2,FRA_ANDROID_FEMALE_19_23,59.0,A
3,USA_IOS_MALE_19_23,54.0,A
4,DEU_ANDROID_FEMALE_24_30,49.0,A


In [21]:
agg_df.groupby("SEGMENT").agg({"PRICE":["mean","max","sum"]})

PRICE                        
              mean        max          sum
SEGMENT                                   
D        27.302596  31.105263  2375.325850
C        32.933339  34.000000  3128.667165
B        35.436170  37.000000  2870.329792
A        41.434736  59.000000  3521.952577

# The segment in which a 33-year-old Turkish woman uses ANDROID and the income she is expected to earn

In [22]:
new_user="TUR_ANDROID_FEMALE_31_40"

In [23]:
agg_df[agg_df["customers_level_based"]==new_user]

,customers_level_based,PRICE,SEGMENT
60,TUR_ANDROID_FEMALE_31_40,39.0,A


# The segment of French women using IOS between the ages of 0-18 and the income they are expected to earn

In [24]:
agg_df[agg_df["customers_level_based"]=="FRA_IOS_FEMALE_0_18"].agg({"PRICE":"mean"})

PRICE    31.312271
dtype: float64